In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas.tools.plotting import scatter_matrix

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.linear_model import LogisticRegression#, LinearRegression

In [3]:
accounts = pd.read_csv('data/MFG-accounts.csv')

In [4]:
customers = pd.read_csv('data/MFG-customers.csv')

In [5]:
invoices = pd.read_csv('data/MFG-invoices.csv')

In [6]:
industry = pd.read_csv('data/MFG-industry.csv')

In [7]:
press_release = pd.read_csv('data/Press_Release.csv')

In [8]:
zip_codes = pd.read_csv('data/zip_code_database.csv')

In [9]:
zip_codes = zip_codes[['zip','state']]

### Customers

In [10]:
customers['Country'] = customers['Country'].replace(['UNITED STATES', 'United States'], ['USA', 'USA',])
customers['Country'] = customers['Country'].fillna('USA')
customers['CustID'] = customers['CustomerID']
customers['State'] = customers['State'].replace('az', 'AZ')

In [11]:
customers['Zip'].unique()

array(['85283', '85206', '92675', '85281', '80120', '85019', '94502',
       '85260', '85027', '85714', '79998', '85282', '85122', '8873',
       '85027-4741', '92618', '85205', '3801', '85044', '90630', '95131',
       '85225', '85040', '7310', '85226', '45147', '85710', '76134',
       '85743', '92612', '85331', '95010', '30024', '84720', '85705',
       '84054', '85043', '85072', '91324', '85308', '87109', '89043', nan,
       '85392', '768765', '85287', '85212', '85006', '85210', '85254',
       '85255', '85621', '85233', '85048', '85053', '28277', '79912',
       '84120', '85013', '86323', '85224', '30677', '85395', '77515',
       '92610', '85018', '85204', '80401', '84119', '80108', 'V5M 4P7',
       '1863', '80421', '85284', '85032', '7724', '85034', '21076-3105',
       '85211', '84321', '86001', '84084', '94063', '95666', '89135',
       '84081', '84341', '94089', '85207', '85250', '85083', '98660',
       '68516', '84116', '85712', '80907', '76051', '85295', '76177',
       

In [12]:
#customers['Zip'] = customers['Zip'].str.replace('-','')#.astype(int)
#customers['Zip'].unique()

In [13]:
customer_region = customers[['CustID', 'State','Zip','Country']]

In [14]:
customer_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 4 columns):
CustID     623 non-null int64
State      612 non-null object
Zip        615 non-null object
Country    623 non-null object
dtypes: int64(1), object(3)
memory usage: 19.5+ KB


### Invoices

In [15]:
invoices['datetime'] = pd.to_datetime(invoices['Date'])

In [16]:
invoices['account_id'] = invoices['Account_ID'].str.replace('-','').astype(int)

In [17]:
invoices['itemid'] = invoices['Item_ID'].str.replace('-','')

In [18]:
invoices['qty'] = invoices['Qty'].str.replace(',','').astype(float)
invoices['qty'] = invoices['qty'].fillna(0)

In [19]:
invoices ['amount'] = (invoices['Amount'].str.replace(',', '').astype(float)).abs()

In [20]:
invoices ['unit_price'] = invoices['Unit_Price'].str.replace(',','').astype(float)
invoices ['unit_price'] = invoices['unit_price'].fillna(0)

In [21]:
invoices['dayofweek'] = invoices['datetime'].dt.dayofweek

In [22]:
repeat=[]
for invoice in invoices.iterrows():
    repeat.append(((invoice[1]['CustID'] == invoices['CustID']) &
    (invoice[1]['datetime'] < invoices['datetime']) &
    (invoice[1]['datetime']  + pd.Timedelta(365, 'd') > invoices['datetime'])).sum()> 0)  

In [23]:
invoices['repeat'] = repeat

In [24]:
def year(col):
    value = int(str(col)[0:4])
    return value
    
def month(col):
    value = int(str(col)[8:11])
    return value

In [25]:
invoices['year'] = invoices['datetime'].apply(year)
invoices['month'] = invoices['datetime'].apply(month)

In [26]:
quarter=[]
for quarter in invoices['month']:
    if quarter>= 1 or quarter<=3:
        quarter=1
    elif quarter >= 4 or quarter <= 6:
        quarter=2
    elif quarter >= 7 or quarter<=9:
        quarter=3
    else:
        quarter=4

In [27]:
invoices['quarter']= quarter

In [28]:
invoices['repeat_c']= invoices['repeat'].replace('True','1').astype(int)
invoices['repeat_c']= invoices['repeat'].replace('False','0').astype(int)

### press_release

In [29]:
press_release['release_date'] = pd.to_datetime(press_release['Release Date'])

In [30]:
press_release ['premium'] = press_release['Premium'].str.replace('-', 'free')
press_release['premium'] = press_release['premium'].fillna('premium')

In [31]:
press_release['headline_len'] = press_release['Headline'].str.len()

### merge invoices industry region

In [32]:
invoices_industry = pd.merge(invoices, industry, how='left', on=['CustID'])

In [33]:
invoices_industry ['industry1'] = invoices_industry['Industry']
invoices_industry ['industry1'] = (invoices_industry['industry1'].replace(['Individual', 'Consumer Product', 'Manufacturing Equipment',
       'Design', 'Antenna', 'Automotive', 'Education', 'Manufacturing',
       'Medical', 'Electronics', 'HVAC', 'Engineering', 'Machinary',
       'Unknown', 'Telecomunications', 'Firearms', 'Swimming Pools',
       'Defence', 'Aerospace', 'Government', 'Injection Molding',
       'Distributor', 'Sensors', 'Testing  ', 'Lighting',
       'Water Treatment', 'Mining', 'Internal', 'Components',
       'Machine Shop', 'Power', 'Alternative Energy', 'Other',
       'Construction', 'Sporting Goods', '3D Printing Service Provider',
       'Semiconductor Equipment', 'Building Products', 'Architecture',
       'Machine shop', 'Clothing', 'Additive Manufacturing',
       'Agriculture Equip', 'Oil & Gas', 'Chemicals', 'Food &Beverage',
       'Turbomachinary', 'Solar', 'Materials', 'Utility', 'Robotics',
       'Furniture', 'Packaging', 'Batteries', 'Printing',
       'Transportation', 'IT'], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
                                31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57]))

#invoices_industry ['industry1'] 

In [34]:
invoices_industry = pd.merge(invoices_industry, customer_region, how='left', on=['CustID'])

In [53]:
invoices_industry.columns

Index(['Date', 'Qty', 'Unit_Price', 'Item_ID', 'Account_ID', 'Amount',
       'CustID', 'datetime', 'account_id', 'itemid', 'qty', 'amount',
       'unit_price', 'dayofweek', 'repeat', 'year', 'month', 'quarter',
       'repeat_c', 'Industry', 'industry1', 'State', 'Zip', 'Country',
       'dummy_state', 'dummy_country'],
      dtype='object')

In [56]:
invoices_industry['Item_ID'].unique()

array(['RP-SLA-1', 'SHP-MFG', 'RP-INJMOLD-1', 'RP-FDM-1', 'RP-SLS-1', nan,
       'RP-OBJ-1', 'RP-RTV-1', 'RP-3DSCAN-1', 'RP-MFG-1', 'RP-MTL0010',
       'RP-MTL0011', 'RP-MTL0015', 'RP-PLS0033', 'RP-PLS0034',
       'RP-PLS0035', 'RP-PLS0036', 'RP-PLS0037', 'RP-PLS0038',
       'RP-PLS0039', 'RP-PLS0040', 'RP-PLS0041', 'RP-PLS0042',
       'RP-PLS0043', 'RP-PLS0028', 'RP-PLS0029', 'RP-PLS0030',
       'RP-FM002868', 'RP-FM002869', 'RP-MTL0014', 'RP-FM002690',
       'RP-1513836', 'RP-1859701', 'RP-PLS0025', 'RP-PLS0026',
       'RP-PLS0027', 'RP-19112-51', 'RP-19112-23', 'RP-19112-24',
       'RP-19112-52', 'RP-19112-57', 'RP-19112-61', 'RP-CS007200 A/B',
       'RP-CS007650', 'RP-19112-25', 'RP-19112-26', 'RP-19112-59',
       'RP-19112-62', 'RP-AMP-1', 'RP-C10047', 'RP-C10048', 'RP-C10049',
       'RP-301550-01', 'RP-301550-02', 'RP-300727A', 'RP-300784A',
       'RP-1-222-0502', 'RP-1-222-0503', 'RP-METAL-1', 'RP-2718001',
       'RP-DLS-1', 'RP-CONSULT-1', 'RP-1001717', 'RP-100171

In [59]:
invoices_industry ['dummy_itemid'] = (invoices_industry['Item_ID'].replace(['RP-SLA-1', 'SHP-MFG',
                                                        'RP-INJMOLD-1', 'RP-FDM-1', 'RP-SLS-1',
       'RP-OBJ-1', 'RP-RTV-1', 'RP-3DSCAN-1', 'RP-MFG-1', 'RP-MTL0010',
       'RP-MTL0011', 'RP-MTL0015', 'RP-PLS0033', 'RP-PLS0034',
       'RP-PLS0035', 'RP-PLS0036', 'RP-PLS0037', 'RP-PLS0038',
       'RP-PLS0039', 'RP-PLS0040', 'RP-PLS0041', 'RP-PLS0042',
       'RP-PLS0043', 'RP-PLS0028', 'RP-PLS0029', 'RP-PLS0030',
       'RP-FM002868', 'RP-FM002869', 'RP-MTL0014', 'RP-FM002690',
       'RP-1513836', 'RP-1859701', 'RP-PLS0025', 'RP-PLS0026',
       'RP-PLS0027', 'RP-19112-51', 'RP-19112-23', 'RP-19112-24',
       'RP-19112-52', 'RP-19112-57', 'RP-19112-61', 'RP-CS007200 A/B',
       'RP-CS007650', 'RP-19112-25', 'RP-19112-26', 'RP-19112-59',
       'RP-19112-62', 'RP-AMP-1', 'RP-C10047', 'RP-C10048', 'RP-C10049',
       'RP-301550-01', 'RP-301550-02', 'RP-300727A', 'RP-300784A',
       'RP-1-222-0502', 'RP-1-222-0503', 'RP-METAL-1', 'RP-2718001',
       'RP-DLS-1', 'RP-CONSULT-1', 'RP-1001717', 'RP-1001718'],
        [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
        31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,
        58,59,60,61,62,63]))


In [61]:
invoices_industry ['dummy_itemid'] = invoices_industry['dummy_itemid'].fillna(90)

In [37]:
invoices_industry ['dummy_state'] = (invoices_industry['State'].replace(['CO', 'AZ',
        'UT', 'CA', 'MA', 'OH', 'NJ', 'PA', 'FL', 'IL',
       'TN', 'NE', 'MD', 'IN', 'WA', 'NC', 'GA', 'NM', 'MI', 'WI', 'NV',
       'NH', 'Te', 'TX', 'KS', 'MN', 'SC', 'MO', 'NS', 'OR', 'VA', 'LA',
       'ON', 'NY', 'BC'],[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
                                31,32,33,34,35]))

In [62]:
invoices_industry ['dummy_state'] = invoices_industry['dummy_state'].fillna(90)

In [63]:
invoices_industry['dummy_country']= (invoices_industry['Country'].replace(['USA', 'Germany', 'Costa Rica', 'China', 'Canada', 'Philippines'],
                                                                          [1,2,3,4,5,6]))

### Logistic regretios


In [65]:
y = invoices_industry['repeat_c'] == True
X = (invoices_industry[['year', 'month', 'unit_price', 'qty', 'account_id', 'amount', 'repeat_c','industry1',
                        'dummy_country','dummy_state','dummy_itemid']])

In [66]:
model1 = LogisticRegression()

In [67]:
model1.fit(X, y)

/Users/lulu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [68]:
model1.coef_

array([[9.50841381e-10, 1.11742988e-11, 2.08516737e-10, 3.59662459e-11,
        9.45049461e-07, 2.50299679e-10, 1.19865378e-12, 4.94438915e-12,
        4.59210498e-13, 3.72408450e-12, 1.03943254e-12]])

In [69]:
#model.predict_proba([[4, 10]])

### Random Fores

In [70]:
from sklearn.model_selection import train_test_split

In [71]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

In [72]:

y = pd.Series(invoices_industry.repeat_c)

X = (invoices_industry [['year', 'month', 'unit_price', 'qty', 'account_id', 'amount', 'industry1',
                         'dummy_country','dummy_state','dummy_itemid']] )

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.33,
                                                    random_state=2)



In [73]:
# Parameter Search                                     
model = DecisionTreeClassifier(max_depth=5)

model.fit(X_train,y_train)
print(model.score(X_test, y_test))

0.8710848842487517


In [74]:
# Parameter Search                                     
model = DecisionTreeClassifier()
depth_parm = np.arange(1, 12, 1)
num_samples_parm = np.arange(5,95,10)
parameters = {'max_depth' : depth_parm,
             'min_samples_leaf' : num_samples_parm}
clf = GridSearchCV(model, parameters, cv=10, n_jobs=-1)
clf.fit(X_train,y_train)
print(clf.score(X_test, y_test))

0.8810712664548344


In [75]:
y_train.mean()

0.8726378172872638

In [76]:
clf.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [77]:
clf.best_params_

{'max_depth': 10, 'min_samples_leaf': 5}

In [78]:
# Train and fit model                                                   
rf = RandomForestClassifier(n_estimators=1000,
                           max_features='auto',
                           random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)
                                     
# Test Prediction
pred = rf.predict(X_test)
print(rf.score(X_test, y_test))

0.9110304130730822
